In [9]:
import torch
from pathlib import Path
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import MarkdownHeaderTextSplitter
import pandas as pd
import json

In [2]:
md_path = Path("bns2023.md")
with open(md_path, "r", encoding="utf-8") as f:
    md_content = f.read()

In [3]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

In [4]:
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(md_content)

In [7]:
chunks = [doc.page_content for doc in md_header_splits]

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer('BAAI/bge-small-en-v1.5', device=device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\HP\.conda\envs\nlp_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
print(f"Generating embeddings for {len(chunks)} chunks...")
embeddings = model.encode(chunks, show_progress_bar=True)

Generating embeddings for 150 chunks...


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
data = []
for i, doc in enumerate(md_header_splits):
    data.append({
        "source_file": md_path.name,
        "content": doc.page_content,
        # Extract headers found by the splitter (e.g., Header 1, Header 2)
        "metadata": json.dumps(doc.metadata), 
        # Convert vector to a list so it's CSV-friendly
        "embedding": json.dumps(embeddings[i].tolist())
    })

In [12]:
df = pd.DataFrame(data)
output_csv = "bns2023_embeddings.csv"
df.to_csv(output_csv, index=False, encoding="utf-8")